### Understanding How MLFlow saves Data

MLFlow has two types of stores

**Backend Store**
- In this store, MLFlow stores metadata information of experiments like parameters, metrics, tags etc
- Bydefault this gets stored in the local filesystem under the name mlruns. In the mlruns file, you will have folder for each experiment. The folder name start with 0 (default experiment) for experiment 1 and if you have two experiments, you will have two folders 0 and 1, these 0 and 1 are experiment ids for the two experiments
- Withing each of these folder you will have a meta.yaml file that contains info about where is the artifact location, the experiment_id, experiment name (for folder 0 experiment name is default) and such
- For each run you create, a unique folder is created in that experiment folder(i.e. folder names with 0 , 1 ...) And inside this run folder, you will have the artifcats folder containing the model, parameters and all
- We can also configure it to store in SQLAlchemy compatible DB (e.g. SQLite, Postgres)

**Artifacts Store**
- Here MLFlow stores all the artifacts like the dataset used for training, the model itself and other configuration files like conda.yaml, requirements.txt that are needed to run the model
- Again by default this is stored in the local filesystem
- We can also configure it to store this information in remote location like Amazon S3 Bucket

### Tracking Experiments with a Local Database

Till now, we have used local files, now we will use local database like sqlite and store the information there

We use the following CLI `mlflow ui --port 8080 --backend-store-uri sqlite:///mlruns.db`

For a custom artifact location, we can use

`mlflow ui --port 8080 --backend-store-uri sqlite:///mlruns.db --default-artifact-root ./artifacts_local`

The above commands are written assuming you are running it from the mlops-learning folder, as the paths to backend and artifacts is given according to that folder

In [36]:
# set the following environment variable
# as we are running in 02-mlfow, using the ../mlruns.db
%env MLFLOW_TRACKING_URI=sqlite:///../mlruns.db

env: MLFLOW_TRACKING_URI=sqlite:///../mlruns.db


../mlruns.db implies that create / use the sqlite db in the parent folder

In [55]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:8080")
mlflow.get_tracking_uri()

'http://127.0.0.1:8080'

In [56]:
mlflow.search_experiments()

[<Experiment: artifact_location='/home/topisano/Desktop/projects/mlops-learning/artifacts_local/0', creation_time=1725114376738, experiment_id='0', last_update_time=1725114376738, lifecycle_stage='active', name='Default', tags={}>]

We can see that the artifact_location is changed to the location that we have specified

We will start logging information

In [57]:
import mlflow

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
# from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

mlflow.set_experiment('mysql-experiment-lr')

db = load_diabetes()
X_train, X_test, y_train, y_test = train_test_split(db.data, db.target)

# Create and train models.
lr = LinearRegression()
lr.fit(X_train, y_train)

# Use the model to make predictions on the test dataset.
predictions = lr.predict(X_test)

rmse = root_mean_squared_error(y_test, predictions)
rmse

2024/08/31 19:56:29 INFO mlflow.tracking.fluent: Experiment with name 'mysql-experiment-lr' does not exist. Creating a new experiment.


np.float64(50.66368785767995)

### Model Signature

**Model Signature**
The Model Signature in MLflow is integral to the clear and accurate operation of models. It defines the expected format for model inputs and outputs, including any additional parameters needed for inference. This specification acts as a definitive guide, ensuring seamless model integration with MLflow’s tools and external services.

**Model Input Example**
Complementing the Model Signature, the Model Input Example gives a concrete instance of what valid model input looks like.

Mlflow's `autolog` automatically inferes the model signature

![](https://mlflow.org/docs/latest/_images/signature-vs-no-signature.png)

Model signatures and input examples are foundational to robust ML workflows, offering a blueprint for model interactions that ensures consistency, accuracy, and ease of use. They act as a contract between the model and its users, providing a definitive guide to the expected data format, thus preventing miscommunication and errors that can arise from incorrect or unexpected inputs.



**Setting a Signature on a Logged Model**

The following example demonstrates how to set a model signature on an already-logged sklearn model. Suppose that you’ve logged a sklearn model without a signature like below:

In [58]:
# saving the model
with mlflow.start_run():
    mlflow.log_metric('rmse',rmse)
    mlflow.sklearn.log_model(lr, artifact_path='lr_model')

2024/08/31 19:56:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/08/31 19:56:36 INFO mlflow.tracking._tracking_service.client: 🏃 View run spiffy-quail-591 at: http://127.0.0.1:8080/#/experiments/1/runs/6d0d7569522a45f4b4a623672d61a2cd.
2024/08/31 19:56:36 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/1.


Here, above, we see that we get a warning stating that we have logged the model without a signature and input_example. To solve this, we will use auto singature

Automatic signature inference (infer_signature) simplifies the process by deducing the schema from provided datasets. It's particularly useful when the input data structure is complex or when rapid prototyping is needed.

In [59]:
from mlflow.models import infer_signature

# it takes the training data and the predictions that model will make on the training data
signature = infer_signature(X_train, lr.predict(X_train))
signature

inputs: 
  [Tensor('float64', (-1, 10))]
outputs: 
  [Tensor('float64', (-1,))]
params: 
  None

We see that model accepts an input of shape (-1, 10) indicating that it will accept 10 features of type float and return an output target of type float. 

Now, there might be a case where you do not have the model and only have the run id. In this case, you can load the model using run id and infer the signature

In [60]:
last_active_run = mlflow.last_active_run()
run_id = last_active_run.info.run_id
run_id

'6d0d7569522a45f4b4a623672d61a2cd'

In [61]:
# load the logged model
model_uri = f"runs:/{run_id}/lr_model" # lr_model is the folder name that we have given while saving the model
model = mlflow.pyfunc.load_model(model_uri)

# construct the model signature from test dataset
signature = infer_signature(X_test, model.predict(X_test))

signature

inputs: 
  [Tensor('float64', (-1, 10))]
outputs: 
  [Tensor('float64', (-1,))]
params: 
  None

After inferring the signature through either methods, now we have to set the signature

In [62]:
from mlflow.models import set_signature
from mlflow.models.model import get_model_info

# set the signature for the logged model
set_signature(model_uri, signature)

# now when you load the model again, it will have the desired signature
assert get_model_info(model_uri).signature == signature

### Model Registry

After we have done all our experimentation, we want to register our model.

Let's say we want to push it to stating, so it may be compared with the production model and if good, then will replace it. So Model Registry is place where you can store the information like what are your Challenger Models(models that are created to replace Production models) and what are your Champion models (models running in production)

When came with a new model. We want to ask some questions. Like what has changed from previous version of model to new version. Is there any preprocessing needed? What are extra libraries that we need to run a new model

And what if when running this new model in production we face some issues and roll back to old model. We need to know where the old model is stored

When doing an ML task, we use the MLFlow Tracking Server to log the parameters, metrics, artifactions and also many different model versions

Once we believe those models are fit for production, then we will "register model" to the MLFlow registry

MLFlow registry is the place where we store the production ready models. So whenver a deployment engineer wants to update the models, they can take a look at the Model Registry to find the new prod ready models

The MLflow Model Registry component is a centralized model store, set of APIs, and UI, to collaboratively manage the full lifecycle of an MLflow Model. It provides model lineage (which MLflow experiment and run produced the model), model versioning, model aliasing, model tagging, and annotations.

Model Registry does not deploy the models, instead it stores the models that are prod ready


In [63]:
# lets register the Linear Regression Model that we have created
result = mlflow.register_model(
    model_uri, "sk-learn-linear-reg"
)


Successfully registered model 'sk-learn-linear-reg'.
2024/08/31 19:56:55 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: sk-learn-linear-reg, version 1
Created version '1' of model 'sk-learn-linear-reg'.


If a registered model with the name doesn’t exist, the method registers a new model, creates Version 1, and returns a ModelVersion MLflow object. If a registered model with the name exists, the method creates a new model version and returns the version object.

We can also register the model directly when logging the model
```python
 # Log the sklearn model and register as version 1
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="sklearn-model",
        signature=signature,
        registered_model_name="sk-learn-linear-model",
    )
```

Note that, while registering, you also need to provide the signature. And if using `mlflow.autolog()`, the signature will be automatically logged

### Interacting with MLFlow Tracking Server

The MlflowClient object allows us to interact with...

- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions.


To instantiate it we need to pass a tracking URI and/or a registry URI

In [64]:
from mlflow import MlflowClient

client = MlflowClient("http://127.0.0.1:8080")

client.search_experiments()

# we can also create a new eperiment
# client.create_experiment(name="my-cool-experiment")

[<Experiment: artifact_location='/home/topisano/Desktop/projects/mlops-learning/artifacts_local/1', creation_time=1725114389950, experiment_id='1', last_update_time=1725114389950, lifecycle_stage='active', name='mysql-experiment-lr', tags={}>,
 <Experiment: artifact_location='/home/topisano/Desktop/projects/mlops-learning/artifacts_local/0', creation_time=1725114376738, experiment_id='0', last_update_time=1725114376738, lifecycle_stage='active', name='Default', tags={}>]

In [47]:
# We can also fetch information from runs
# from mlflow.entities import ViewType

# runs = client.search_runs(
#     experiment_ids='1',
#     filter_string="metrics.rmse < 7",
#     run_view_type=ViewType.ACTIVE_ONLY,
#     max_results=5,
#     order_by=["metrics.rmse ASC"]
# )

# for run in runs:
#     print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

Searching for Registered Models

In [65]:
from pprint import pprint

# looping through registered model
for rm in client.search_registered_models():
    pprint(dict(rm), indent=4)

{   'aliases': {},
    'creation_timestamp': 1725114415752,
    'description': '',
    'last_updated_timestamp': 1725114415783,
    'latest_versions': [   <ModelVersion: aliases=[], creation_timestamp=1725114415783, current_stage='None', description='', last_updated_timestamp=1725114415783, name='sk-learn-linear-reg', run_id='6d0d7569522a45f4b4a623672d61a2cd', run_link='', source='/home/topisano/Desktop/projects/mlops-learning/artifacts_local/1/6d0d7569522a45f4b4a623672d61a2cd/artifacts/lr_model', status='READY', status_message='', tags={}, user_id='', version='1'>],
    'name': 'sk-learn-linear-reg',
    'tags': {}}


Updating a Registered Model

In [66]:
# updating a model
client.update_model_version(
    name="sk-learn-linear-reg",
    version=1,
    description="This model version is a scikit-learn linear regression",
)

<ModelVersion: aliases=[], creation_timestamp=1725114415783, current_stage='None', description='This model version is a scikit-learn linear regression', last_updated_timestamp=1725114430728, name='sk-learn-linear-reg', run_id='6d0d7569522a45f4b4a623672d61a2cd', run_link='', source='/home/topisano/Desktop/projects/mlops-learning/artifacts_local/1/6d0d7569522a45f4b4a623672d61a2cd/artifacts/lr_model', status='READY', status_message='', tags={}, user_id='', version='1'>

Creating a Registered Mode

In [50]:
# from mlflow import MlflowClient

# client = MlflowClient()

# # create an empty registered model with no version
# client.create_registered_model("sk-learn-random-forest-reg-model")

# # assign that registered model to an existing source
# result = client.create_model_version(
#     name="sk-learn-random-forest-reg-model",
#     source="mlruns/0/d16076a3ec534311817565e6527539c0/artifacts/sklearn-model",
#     run_id="d16076a3ec534311817565e6527539c0",
# )

Getting a Registered Model and Printing its Info

In [67]:
def print_model_info(rm):
    print("--Model--")
    print("name: {}".format(rm.name))
    print("aliases: {}".format(rm.aliases))

model = client.get_registered_model('sk-learn-linear-reg')
print_model_info(model)

--Model--
name: sk-learn-linear-reg
aliases: {}


### Deploying and Organize Models with Aliases and Tags

Model aliases and tags help you deploy and organize your models in the Model Registry.

For eample, we can set an alias called Champion to the model that is running in the production. And we can set an alias called Challenger to the model that is newly created and is ready for production

So now data scientist can evaluate this Challenger model and check if it performs better than the Champion. And if its, then we remove the Champion alias to the old model and put the Champion alias to the new model

For these Taks, we deal with MLFlowClient object

Indepth Information

**Model Version**
- Each registered model can have one or many versions. When a new model is added to the Model Registry, it is added as version 1. Each new model registered to the same model name increments the version number. Model versions have tags, which can be useful for tracking attributes of the model version (e.g. pre_deploy_checks: “PASSED”)

**Model Alias**
- Model aliases allow you to assign a mutable, named reference to a particular version of a registered model. By assigning an alias to a specific model version, you can use the alias to refer that model version via a model URI or the model registry API. For example, you can create an alias named champion that points to version 1 of a model named MyModel. You can then refer to version 1 of MyModel by using the URI models:/MyModel@champion.

- Aliases are especially useful for deploying models. For example, you could assign a champion alias to the model version intended for production traffic and target this alias in production workloads. You can then update the model serving production traffic by reassigning the champion alias to a different model version.

**Tags**
- Tags are key-value pairs that you associate with registered models and model versions, allowing you to label and categorize them by function or status. For example, you could apply a tag with key "task" and value "question-answering" (displayed in the UI as task:question-answering) to registered models intended for question answering tasks. At the model version level, you could tag versions undergoing pre-deployment validation with validation_status:pending and those cleared for deployment with validation_status:approved.



Let's set tags to the model

In [68]:
# setting tags for the registered model
client.set_registered_model_tag("sk-learn-linear-reg", "task", "regression")
# now check the UI for that tag

In [69]:
# deleting tags
client.delete_registered_model_tag("sk-learn-linear-reg", "task")
# if you check the UI, the tag will be deleted

Let's create Aliases to the model

In [70]:
# Here, we set our model as Champion
client.set_registered_model_alias("sk-learn-linear-reg", "champion", '1')

In [71]:
# get a model version by alias
client.get_model_version_by_alias("sk-learn-linear-reg", "champion")


<ModelVersion: aliases=['champion'], creation_timestamp=1725114415783, current_stage='None', description='This model version is a scikit-learn linear regression', last_updated_timestamp=1725114430728, name='sk-learn-linear-reg', run_id='6d0d7569522a45f4b4a623672d61a2cd', run_link='', source='/home/topisano/Desktop/projects/mlops-learning/artifacts_local/1/6d0d7569522a45f4b4a623672d61a2cd/artifacts/lr_model', status='READY', status_message='', tags={}, user_id='', version='1'>

**Fetching an MLflow Model from the Model Registry**

In [72]:
# here we are using pyfunction approach to fetch the model
import mlflow.pyfunc

model_name = "sk-learn-linear-reg"
model_version = 1

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

y_pred = model.predict(X_test)
root_mean_squared_error(y_test, y_pred)

np.float64(50.66368785767995)

In [73]:
model

mlflow.pyfunc.loaded_model:
  artifact_path: lr_model
  flavor: mlflow.sklearn
  run_id: 6d0d7569522a45f4b4a623672d61a2cd